In [1]:
from mpi4py import MPI
from dolfinx import mesh
domain = mesh.create_unit_square(comm=MPI.COMM_WORLD, nx=10, ny=10, cell_type=mesh.CellType.quadrilateral)

In [2]:
from dolfinx import fem
V = fem.functionspace(domain, ("Lagrange", 1))

In [3]:
uD = fem.Function(V)
uD.interpolate(lambda x: 1 + x[0]**2 + 2*x[1]**2)

In [4]:
uD

Coefficient(FunctionSpace(Mesh(blocked element (Basix element (P, quadrilateral, 1, gll_warped, unset, False, float64, []), (2,)), 0), Basix element (P, quadrilateral, 1, gll_warped, unset, False, float64, [])), 0)

In [5]:
import numpy
# Create facet to cell connectivity required to determine boundary facets
tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(fdim, tdim)
boundary_facets = mesh.exterior_facet_indices(domain.topology)

In [6]:
boundary_facets

array([  0,   1,   3,   5,   9,  13,  17,  23,  27,  35,  39,  49,  53,
        65,  69,  83,  87, 103, 107, 125, 128, 129, 146, 147, 148, 163,
       164, 177, 178, 189, 190, 199, 200, 207, 208, 213, 214, 217, 218,
       219], dtype=int32)

In [7]:
boundary_dofs = fem.locate_dofs_topological(V, fdim, boundary_facets)
bc = fem.dirichletbc(uD, boundary_dofs)


In [8]:
import ufl
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)


In [9]:
from dolfinx import default_scalar_type
f = fem.Constant(domain, default_scalar_type(-6))


Now, all problem variables are defined.

In [10]:
#weak form
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = f * v * ufl.dx


In [11]:
from dolfinx.fem.petsc import LinearProblem
problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [12]:
uh

Coefficient(FunctionSpace(Mesh(blocked element (Basix element (P, quadrilateral, 1, gll_warped, unset, False, float64, []), (2,)), 0), Basix element (P, quadrilateral, 1, gll_warped, unset, False, float64, [])), 1)

compute the L2 error

In [13]:
V2 = fem.functionspace(domain, ("Lagrange", 2))
uex = fem.Function(V2)
uex.interpolate(lambda x: 1 + x[0]**2 + 2 * x[1]**2)
L2_error = fem.form(ufl.inner(uh - uex, uh - uex) * ufl.dx)
error_local = fem.assemble_scalar(L2_error)
error_L2 = numpy.sqrt(domain.comm.allreduce(error_local, op=MPI.SUM))
error_max = numpy.max(numpy.abs(uD.x.array-uh.x.array))
# Only print the error on one process
if domain.comm.rank == 0:
    print(f"Error_L2 : {error_L2:.2e}")
    print(f"Error_max : {error_max:.2e}")


Error_L2 : 5.27e-03
Error_max : 4.44e-15


In [14]:
import pyvista
print(pyvista.global_theme.jupyter_backend)


trame


In [ ]:
from dolfinx import plot
pyvista.start_xvfb()
domain.topology.create_connectivity(tdim, tdim)
topology, cell_types, geometry = plot.vtk_mesh(domain, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)


In [16]:
plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("fundamentals_mesh.png")


Widget(value='<iframe src="http://localhost:46309/index.html?ui=P_0x7d5c148b1400_0&reconnect=auto" class="pyvi…

In [17]:
u_topology, u_cell_types, u_geometry = plot.vtk_mesh(V)

In [18]:
u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.point_data["u"] = uh.x.array.real
u_grid.set_active_scalars("u")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=True)
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()

Widget(value='<iframe src="http://localhost:46309/index.html?ui=P_0x7d5c114d5810_1&reconnect=auto" class="pyvi…